**Sensitivity Analysis**

- Test alternative priors (HalfNormal, Student-t, more informative Normal)

- Check stability of posterior estimates

- Figures / Tables:

- 
    - Prior vs posterior sensitivity comparison

    - Posterior distributions for different priors

In [ ]:
# ============================================================
# Sensitivity Analysis
# ============================================================

import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv("data/processed/pbc_clean.csv")

df_baseline = (
    df.sort_values(["id", "year"])
      .groupby("id", as_index=False)
      .first()
)

T = df_baseline["years"].values
E = df_baseline["status2"].values

covariates = ["age", "sex", "drug", "serBilir", "albumin", "edema"]
X = df_baseline[covariates].values
N, P = X.shape

In [ ]:
priors = {
    "Normal(0,1)": lambda: pm.Normal("alpha", 0, 1),
    "Normal(0,5)": lambda: pm.Normal("alpha", 0, 5),
    "HalfNormal(1)": lambda: pm.HalfNormal("alpha", sigma=1),
    "StudentT(3,0,1)": lambda: pm.StudentT("alpha", nu=3, mu=0, sigma=1)
}

In [ ]:
n_draws = 1000
n_tune = 500
results = {}

for prior_name, prior_fn in priors.items():
    print(f"\nRunning model with {prior_name} prior...")

    with pm.Model() as model:

        # Alpha
        alpha = prior_fn()
        
        # Beta
        if "StudentT" in prior_name:
            beta = pm.StudentT("beta", nu=3, mu=0, sigma=1, shape=P)
        elif "HalfNormal" in prior_name:
            beta = pm.HalfNormal("beta", sigma=1, shape=P)
        else:
            sigma = 1 if "Normal(0,1)" in prior_name else 5
            beta = pm.Normal("beta", mu=0, sigma=sigma, shape=P)

        # Log-lambda
        log_lambda = alpha + pm.math.dot(X, beta)
        lambda_ = pm.math.exp(log_lambda)

        # Likelihood (Exponential)
        loglik = E * (pm.math.log(lambda_) - lambda_ * T) + (1 - E) * (-lambda_ * T)
        pm.Potential("likelihood", loglik.sum())

        # Sample
        trace = pm.sample(
            draws=n_draws,
            tune=n_tune,
            chains=2,
            target_accept=0.9,
            return_inferencedata=True,
            progressbar=False
        )

        results[prior_name] = trace

In [ ]:
summary_dfs = []

for prior_name, trace in results.items():
    summary = az.summary(trace, var_names=["beta"], hdi_prob=0.95)
    summary["Prior"] = prior_name
    summary["Covariate"] = covariates
    summary_dfs.append(summary.reset_index(drop=True))

df_summary = pd.concat(summary_dfs)
df_summary.to_csv("results/tables/sensitivity_posteriors.csv", index=False)

In [ ]:
plt.figure(figsize=(8, 6))

for cov in covariates:
    subset = df_summary[df_summary["Covariate"] == cov]
    for _, row in subset.iterrows():
        plt.errorbar(
            x=row["Prior"], 
            y=row["mean"], 
            yerr=[[row["mean"] - row["hdi_2.5%"]], [row["hdi_97.5%"] - row["mean"]]],
            fmt="o", capsize=4
        )

plt.xticks(rotation=45)
plt.axhline(0, color="red", linestyle="--")
plt.ylabel("Posterior Mean of β")
plt.title("Sensitivity of Posterior β under Alternative Priors")
plt.tight_layout()
plt.savefig("results/figures/sensitivity_beta_comparison.png", dpi=300)
plt.close()

In [ ]:
import matplotlib

plt.figure(figsize=(6,6))
angles = np.linspace(0, 2*np.pi, len(covariates), endpoint=False)
angles = np.concatenate((angles, [angles[0]]))  # close the circle

for prior_name, trace in results.items():
    beta_samples = trace.posterior["beta"].mean(dim=["chain", "draw"]).values
    beta_samples = np.concatenate((beta_samples, [beta_samples[0]]))
    plt.plot(angles, beta_samples, label=prior_name)

plt.xticks(angles[:-1], covariates)
plt.title("Radar Plot: Posterior β under Different Priors")
plt.legend()
plt.tight_layout()
plt.savefig("results/figures/sensitivity_radar_plot.png", dpi=300)
plt.close()